### Chargement d'un graphe depuis un fichier

In [1]:
# Ajouter le chemin vers le dossier src
import sys
import os
sys.path.append(os.path.abspath("../src"))

# Tous les imports nécessaires
from dotenv import load_dotenv
import random
from graph import Graph

# Charger les variables d'environnement
load_dotenv()

# Récupérer les chemins depuis le fichier .env
geonames_path = os.getenv("GEONAMES_PATH")
shapefile_path = os.getenv("SHAPEFILE_PATH")
output_path = os.getenv("OUTPUT_PATH")
serialized_graph_path = os.getenv("SERIALIZE_GRAPH_PATH")

In [2]:
graph_path = "../data/datasets/size_200/graph_size200_density0.01.pkl"

# Charger le graphe
try:
    graph = Graph.load(graph_path)
    print("Graphe chargé avec succès.")
except FileNotFoundError:
    print(f"Le fichier {graph_path} est introuvable.")
except Exception as e:
    print(f"Erreur lors du chargement du graphe : {e}")

Graph deserialized from ../data/datasets/size_200/graph_size200_density0.01.pkl
Graphe chargé avec succès.


### Enregistrement d'un graphe en .svg

In [ ]:
savePath = f"../data/results/graph_{random.randint(1, 9999)}.svg"

try:
    graph.save_graph_svg(savePath)
    print(f"Graphe enregistré avec succès au format SVG : {savePath}")
except Exception as e:
    print(f"Erreur lors de l'enregistrement du graphe en SVG : {e}")

Graphe enregistré avec succès au format SVG : ../data/results/graph_5027.svg
